In [233]:
import pandas as pd
import numpy as np
import acessos as ac
import os
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import seaborn as sns

In [234]:
data_inicio = '2023-01-01'
data_fim = '2023-07-01'

#### Trazendo dados geral

In [235]:
query_cx = f"""

select 
  dt_mes,
  hierarquia_1, 
  hierarquia_4, 
  count(distinct cpf_customer) as clientes_cx
from 
  (
    SELECT 
      date_trunc(
        'month', 
        cast(c.dt_chat as date)
      ) as dt_mes, 
      c.cpf_customer
      , 
      max(nm_tag_hierarquia_1) as hierarquia_1, 
      max(nm_tag_hierarquia_4) as hierarquia_4 
    FROM 
      cx_curated_zone.cx_contact_rate c 
      LEFT JOIN cx_curated_zone.helpshift_issues AS issues_macro ON c.id_issue = issues_macro.id_issue 
      LEFT JOIN cx_curated_zone.helpshift_tag_niveis AS helpshift_tag ON c.id_issue = helpshift_tag.id_issue 
      LEFT JOIN processed_zone_api_cxm_tags.tags_hierarquia_gerencial as novas_tags ON trim(
        lower(n3)
      ) = trim(
        lower(nm_tag_n3)
      ) 
    WHERE 
      dt_chat >= to_date('{data_inicio}', 'yyyy-mm-dd') 
      and dt_chat < to_date('{data_fim}', 'yyyy-mm-dd') 
      AND c.id_issue is not null 
      and nm_tag_hierarquia_1 <> 'Crédito e Cobrança' 
      AND issues_macro.ds_entidade_resolucao = 'agente' 
    GROUP BY 
      1, 
      2
  ) 
group by 
  1, 
  2,
  3


"""

In [236]:
df_count_cx = ac.df_athena_q('flavia-costa', query_cx)

#### Trazendo Recorte

In [237]:
query_recorte = f"""

WITH chats AS (
  SELECT 
    cpf_customer, 
    count(DISTINCT id_issue) qtd_chats 
  FROM 
    cx_curated_zone.helpshift_issues 
  WHERE 
    dt_criacao_chat >= to_date('2022-07-01', 'yyyy-mm-dd') 
    AND dt_criacao_chat < to_date('{data_fim}', 'yyyy-mm-dd') 
  GROUP BY 
    1
), 
chats_recorte_detalhado AS (
  SELECT 
    date_trunc(
      'month', 
      cast(c.dt_chat as date)
    ) as dt_mes,
    (nm_tag_hierarquia_1) as hierarquia_1, 
    (nm_tag_hierarquia_4) as hierarquia_4, 
    c.cpf_customer, 
    max(
      case when tipo_de_tag = 'Problema' then 1 else 0 end
    ) as flag_problema 
    , 
    max(nr_tpr) as nr_tpr, 
    max(nr_tma) as nr_tma, 
    max(nr_tmat) as nr_tmat
  FROM 
    cx_curated_zone.cx_contact_rate c 
    LEFT JOIN cx_curated_zone.helpshift_issues AS issues_macro ON c.id_issue = issues_macro.id_issue 
    LEFT JOIN cx_curated_zone.helpshift_tag_niveis AS helpshift_tag ON c.id_issue = helpshift_tag.id_issue 
    LEFT JOIN processed_zone_api_cxm_tags.tags_hierarquia_gerencial as novas_tags ON trim(
      lower(n3)
    ) = trim(
      lower(nm_tag_n3)
    ) 
    LEFT JOIN chats AS cht ON cht.cpf_customer = c.cpf_customer 
  WHERE 
    dt_chat >= to_date('{data_inicio}', 'yyyy-mm-dd') 
    and dt_chat < to_date('{data_fim}', 'yyyy-mm-dd') 
    AND c.id_issue is not null 
    and nm_tag_hierarquia_1 <> 'Crédito e Cobrança' 
    AND issues_macro.ds_entidade_resolucao = 'agente' --RECORTE
    and cht.cpf_customer is not NULL 
    AND cht.qtd_chats < 4 
    and issues_macro.nr_tmat > 90 
  GROUP BY 
    1, 
    2,
    3,
    4
), 
de_para AS (
  SELECT 
    DISTINCT dt_mes, 
    date_add('day', -91, dt_mes) as periodo_antes_inicio, 
    date_add('day', -1, dt_mes) as periodo_antes_fim, 
    date_add('month', 1, dt_mes) as periodo_depois_inicio, 
    date_add('month', 3, dt_mes) as periodo_depois_fim, 
    date_add('month', 2, dt_mes) as periodo_depois30_fim 
  FROM 
    chats_recorte_detalhado
), 
spending_antes_total_ AS (
  SELECT 
    distinct s.cpf, 
    s.id_autorizacao, 
    c.dt_mes, 
    periodo_antes_inicio, 
    periodo_antes_fim, 
    periodo_depois_inicio, 
    periodo_depois_fim, 
    periodo_depois30_fim, 
    s.vl_transacao as vl_compra_antes_90d --, count(distinct s.id_autorizacao) as qt_compra_antes_90d
    , 
    bc.dt_criacao_conta as dt_criacao_conta, 
    s.dt_dia as dt_last_use_credit, 
    a.dt_first_use_credit as dt_first_use_credit, 
    gc.vl_current_limit as vl_current_limit 
  FROM 
    platform_curated_zone.transaction_dedicada_spending s 
    INNER JOIN de_para c ON (
      cast(s.dt_dia as date) >= c.periodo_antes_inicio 
      AND cast(s.dt_dia as date) <= c.periodo_antes_fim
    ) --IGUAL OU MAIOR E IGUAL
    LEFT JOIN (
      SELECT 
        cpf, 
        id_customer, 
        min(cd_yearmonth) as dt_criacao_conta 
      FROM 
        customer_curated_zone.ca_book_cliente 
      GROUP BY 
        1, 
        2
    ) bc ON (bc.cpf = s.cpf) 
    LEFT JOIN customer_curated_zone.ca_book_status_cliente a ON (a.id_customer = bc.id_customer) --limite
    LEFT JOIN customer_curated_zone.ca_book_gestao_carteira gc ON (
      s.cpf = gc.cpf 
      AND cast(
        replace(
          substring(
            cast(c.dt_mes as varchar), 
            1, 
            7
          ), 
          '-', 
          ''
        ) as double
      ) = gc.cd_yearmonth
    ) 
  WHERE 
    s.vl_transacao > 0 
    AND bc.dt_criacao_conta IS NOT NULL 
    AND gc.vl_current_limit IS NOT NULL
), 
spending_antes_total AS (
  select 
    cpf, 
    dt_mes, 
    periodo_antes_inicio, 
    periodo_antes_fim, 
    periodo_depois_fim, 
    periodo_depois30_fim, 
    periodo_depois_inicio, 
    dt_criacao_conta, 
    sum(vl_compra_antes_90d) as vl_compra_antes_90d, 
    count(distinct id_autorizacao) as qt_compra_antes_90d, 
    max(dt_last_use_credit) as dt_last_use_credit, 
    min(dt_first_use_credit) as dt_first_use_credit, 
    max(vl_current_limit) as vl_current_limit 
  from 
    spending_antes_total_ 
  group by 
    1, 
    2, 
    3, 
    4, 
    5, 
    6, 
    7, 
    8
), 
base_total_add_chats AS (
  SELECT 
    s.cpf, 
    s.dt_mes as dt_mes_true, 
    s.periodo_antes_inicio, 
    s.periodo_antes_fim, 
    s.periodo_depois_inicio, 
    s.periodo_depois_fim, 
    s.periodo_depois30_fim, 
    s.vl_compra_antes_90d, 
    s.qt_compra_antes_90d, 
    s.dt_last_use_credit, 
    s.dt_first_use_credit, 
    s.dt_criacao_conta, 
    s.vl_current_limit, 
    c.*, 
    case when c.cpf_customer is null then 0 else 1 end as usou_chat 
  FROM 
    chats_recorte_detalhado c 
    LEFT JOIN spending_antes_total s on (
      s.cpf = c.cpf_customer 
      and c.dt_mes = s.dt_mes
    )
), 
spending_depois_30 as (
  select 
    c.cpf, 
    c.dt_mes_true, 
    sum(s_30.vl_transacao) as vl_compra_depois_30d, 
    count(distinct s_30.id_autorizacao) as qt_compras_depois_30d 
  from 
    base_total_add_chats c 
    left join platform_curated_zone.transaction_dedicada_spending s_30 on (
      c.cpf = s_30.cpf 
      and cast(s_30.dt_dia as date) >= c.periodo_depois_inicio 
      and cast(s_30.dt_dia as date) <= c.periodo_depois30_fim
    ) 
  group by 
    1, 
    2
), 
spending_depois_60 as (
  select 
    c.cpf, 
    c.dt_mes_true, 
    sum(s_60.vl_transacao) as vl_compra_depois_60d, 
    count(distinct s_60.id_autorizacao) as qt_compras_depois_60d 
  from 
    base_total_add_chats c 
    left join platform_curated_zone.transaction_dedicada_spending s_60 on (
      c.cpf = s_60.cpf 
      and cast(s_60.dt_dia as date) >= c.periodo_depois_inicio 
      and cast(s_60.dt_dia as date) <= c.periodo_depois_fim
    ) 
  group by 
    1, 
    2
), 
union_ltv as (
  select 
    distinct mesref, 
    tipo, 
    dias_atraso, 
    cpf, 
    spending_credito, 
    rec_itr, 
    rec_recarga, 
    rec_demais_encargos, 
    desp_pdd, 
    rec_juros, 
    custos, 
    ltv 
  from 
    public.rentabilidade_cartoes_diego_camilo
) 
SELECT 
  a.cpf, 
  a.dt_mes_true, 
  a.periodo_antes_inicio, 
  a.periodo_antes_fim, 
  a.periodo_depois_inicio, 
  a.periodo_depois_fim, 
  a.periodo_depois30_fim, 
  a.qt_compra_antes_90d, 
  a.dt_last_use_credit, 
  a.vl_current_limit, 
  a.flag_problema, 
  a.nr_tpr, 
  a.nr_tma, 
  a.nr_tmat, 
  a.usou_chat, 
  a.hierarquia_1, 
  a.hierarquia_4, 
  max(a.vl_compra_antes_90d) vl_compra_antes_90d, 
  min(a.dt_first_use_credit) dt_first_use_credit, 
  max(gro_.vl_renda_declarada) vl_renda_declarada, 
  min(a.dt_criacao_conta) dt_criacao_conta,
  min(gro_.dt_opening_account) as dt_opening_account,
  count(
    distinct substring(
      cast(dt_event as varchar), 
      1, 
      10
    )
  ) as dias_uso_app, 
  max(d1.vl_compra_depois_30d) vl_compra_depois_30d, 
  max(d1.qt_compras_depois_30d) qt_compras_depois_30d, 
  max(d2.vl_compra_depois_60d) vl_compra_depois_60d, 
  max(d2.qt_compras_depois_60d) qt_compras_depois_60d, 
  max(l.spending_credito) as spending_ltv, 
  max(l.rec_itr) as rec_itr, 
  max(l.rec_recarga) as rec_recarga, 
  max(l.rec_demais_encargos) as rec_demais_encargos, 
  max(l.desp_pdd) as desp_pdd, 
  max(l.rec_juros) as rec_juros, 
  max(l.custos) as custos, 
  max(l.ltv) as ltv, 
  max(l.dias_atraso) as dias_atraso_ltv 
FROM 
  base_total_add_chats a 
  LEFT JOIN customer_curated_zone.ca_book_cliente b_c ON (a.cpf = b_c.cpf) 
  LEFT JOIN customer_curated_zone.ca_book_growth gro_ ON gro_.id_customer = b_c.id_customer 
  LEFT JOIN customer_curated_zone.ca_analitico_geoloc ap on (
    ap.cpf = a.cpf 
    and ap.dt_event >= a.periodo_antes_inicio 
    and ap.dt_event <= a.periodo_antes_fim
  ) 
  LEFT JOIN spending_depois_30 d1 on (
    a.cpf = d1.cpf 
    and a.dt_mes_true = d1.dt_mes_true
  ) 
  LEFT JOIN spending_depois_60 d2 on (
    a.cpf = d2.cpf 
    and a.dt_mes_true = d2.dt_mes_true
  ) 
  left join union_ltv l on (
    a.cpf = l.cpf 
    and a.dt_mes_true = cast(
      concat(l.mesref, '-01') as date
    )
  ) 
where 
  a.usou_chat = 1 
group by 
  1, 
  2, 
  3, 
  4, 
  5, 
  6, 
  7, 
  8, 
  9, 
  10, 
  11, 
  12, 
  13, 
  14, 
  15, 
  16, 
  17

"""

In [238]:
df = ac.df_athena_q('flavia-costa', query_recorte)

In [239]:
len(df)

427495

In [240]:
missing_zero = ['qt_compra_antes_90d', 'vl_compra_antes_90d', 'vl_compra_depois_60d', 'spending_ltv', 'ltv', 'dias_atraso_ltv', 'rec_itr',
					'rec_recarga', 'rec_demais_encargos', 'desp_pdd', 'rec_juros', 'custos']

for column in missing_zero:
    df[column].fillna(0, inplace = True)

Criação de variáveis

In [241]:
df['dt_opening_account'] = pd.to_datetime(df['dt_opening_account'], format='%Y-%m-%d')
df['dt_mes_true'] = pd.to_datetime(df['dt_mes_true'], format='%Y-%m-%d')
df['cc_time'] = (df['dt_mes_true'] - df['dt_opening_account'])/ np.timedelta64(1, 'D')
    
df['dt_first_use_credit'] = pd.to_datetime(df['dt_first_use_credit'], format='%Y-%m-%d')
df['dias_ativacao'] = (df['dt_mes_true'] - df['dt_first_use_credit'])/ np.timedelta64(1, 'D')
       
df['dt_last_use_credit'] = pd.to_datetime(df['dt_last_use_credit'], format='%Y-%m-%d', errors = 'coerce')
df['dias_desde_ultima_compra'] = (df['dt_mes_true'] - df['dt_last_use_credit']).dt.days
   
df['vl_compra_antes_90d_log'] = np.log(df.vl_compra_antes_90d) #Ajuste da variável


/home/flavia.costa/.local/lib/python3.8/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


## Lidando com missings

In [242]:
for column in df.columns:
    if df[column].dtype == 'object':
        df[column].fillna(df[column].mode()[0], inplace = True)
    elif column == 'vl_compra_antes_90d_log':
        df[column].fillna(0, inplace = True)
    elif df[column].dtype != 'object':
        df[column].fillna(df[column].median(), inplace = True)

Função de normalização

In [243]:
def normalizando(x, var_min, var_max):
    y = (x - var_min) / (var_max - var_min)
    return y

In [244]:
df_n = pd.DataFrame()

df_n['cc_time'] = normalizando(df['cc_time'], 0.19, 2062.09)
df_n['vl_current_limit'] = normalizando(df['vl_current_limit'], 0, 12300)
df_n['dias_uso_app'] = normalizando(df['dias_uso_app'], 0, 35)
df_n['vl_renda_declarada'] = normalizando(df['vl_renda_declarada'], 0, 999999.99)
df_n['dias_ativacao'] = normalizando(df['dias_ativacao'], 0, 1812.25)
df_n['qt_compra_antes_90d'] = normalizando(df['qt_compra_antes_90d'], 1, 78)
df_n['dias_desde_ultima_compra'] = normalizando(df['dias_desde_ultima_compra'], 1, 77)


## Base ano anterior

In [245]:
query_aa = f"""

WITH chats AS (
  SELECT 
    cpf_customer, 
    count(DISTINCT id_issue) qtd_chats 
  FROM 
    cx_curated_zone.helpshift_issues 
  WHERE 
    dt_criacao_chat >= to_date('2021-01-01', 'yyyy-mm-dd') 
    AND dt_criacao_chat < to_date('{data_inicio}', 'yyyy-mm-dd') 
  GROUP BY 
    1
), 
chats_agg AS (
  SELECT 
    date_trunc(
      'month', 
      cast(c.dt_chat as date)
    ) as dt_mes, 
    c.cpf_customer -- problema?
    , 
    max(
      case when tipo_de_tag = 'Problema' then 1 else 0 end
    ) as flag_problema --finalizado por automacao
    , 
    max(nr_tpr) as nr_tpr, 
    max(nr_tma) as nr_tma, 
    max(nr_tmat) as nr_tmat, 
    max(nm_tag_hierarquia_1) as hierarquia_1 
  FROM 
    cx_curated_zone.cx_contact_rate c 
    LEFT JOIN cx_curated_zone.helpshift_issues AS issues_macro ON c.id_issue = issues_macro.id_issue 
    LEFT JOIN cx_curated_zone.helpshift_tag_niveis AS helpshift_tag ON c.id_issue = helpshift_tag.id_issue 
    LEFT JOIN processed_zone_api_cxm_tags.tags_hierarquia_gerencial as novas_tags ON trim(
      lower(n3)
    ) = trim(
      lower(nm_tag_n3)
    ) 
    INNER JOIN chats AS cht ON cht.cpf_customer = c.cpf_customer 
    AND cht.qtd_chats < 4 
  WHERE 
    dt_chat >= to_date('2022-01-01', 'yyyy-mm-dd') 
    and dt_chat < to_date('{data_inicio}', 'yyyy-mm-dd') 
    AND c.id_issue is not null 
    and issues_macro.nr_tmat > 90 
    and nm_tag_hierarquia_1 <> 'Crédito e Cobrança' 
  GROUP BY 
    1, 
    2
), 
de_para AS (
  SELECT 
    DISTINCT dt_mes, 
    date_add('day', -91, dt_mes) as periodo_antes_inicio, 
    date_add('day', -1, dt_mes) as periodo_antes_fim, 
    date_add('month', 9, dt_mes) as periodo_depois_inicio, 
    date_add('month', 12, dt_mes) as periodo_depois_fim 
  FROM 
    chats_agg
), 
spending_antes_total_ AS (
  SELECT 
    distinct s.cpf, 
    s.id_autorizacao, 
    c.dt_mes, 
    periodo_antes_inicio, 
    periodo_antes_fim, 
    periodo_depois_inicio, 
    periodo_depois_fim, 
    s.vl_transacao as vl_compra_antes_90d --, count(distinct s.id_autorizacao) as qt_compra_antes_90d
    , 
    bc.dt_criacao_conta as dt_criacao_conta, 
    s.dt_dia as dt_last_use_credit, 
    a.dt_first_use_credit as dt_first_use_credit, 
    gc.vl_current_limit as vl_current_limit 
  FROM 
    platform_curated_zone.transaction_dedicada_spending s 
    INNER JOIN de_para c ON (
      cast(s.dt_dia as date) >= c.periodo_antes_inicio 
      AND cast(s.dt_dia as date) <= c.periodo_antes_fim
    ) --IGUAL OU MAIOR E IGUAL
    LEFT JOIN (
      SELECT 
        cpf, 
        id_customer, 
        min(cd_yearmonth) as dt_criacao_conta 
      FROM 
        customer_curated_zone.ca_book_cliente 
      GROUP BY 
        1, 
        2
    ) bc ON (bc.cpf = s.cpf) 
    LEFT JOIN customer_curated_zone.ca_book_status_cliente a ON (a.id_customer = bc.id_customer) --limite
    LEFT JOIN customer_curated_zone.ca_book_gestao_carteira gc ON (
      s.cpf = gc.cpf 
      AND cast(
        replace(
          substring(
            cast(c.dt_mes as varchar), 
            1, 
            7
          ), 
          '-', 
          ''
        ) as double
      ) = gc.cd_yearmonth
    ) 
  WHERE 
    s.vl_transacao > 0 
    AND bc.dt_criacao_conta IS NOT NULL 
    AND gc.vl_current_limit IS NOT NULL
), 
spending_antes_total AS (
  select 
    cpf, 
    dt_mes, 
    periodo_antes_inicio, 
    periodo_antes_fim, 
    periodo_depois_fim, 
    periodo_depois_inicio, 
    dt_criacao_conta, 
    sum(vl_compra_antes_90d) as vl_compra_antes_90d, 
    count(distinct id_autorizacao) as qt_compra_antes_90d, 
    max(dt_last_use_credit) as dt_last_use_credit, 
    min(dt_first_use_credit) as dt_first_use_credit, 
    max(vl_current_limit) as vl_current_limit 
  from 
    spending_antes_total_ 
  group by 
    1, 
    2, 
    3, 
    4, 
    5, 
    6, 
    7
), 
base_total_add_chats AS (
  SELECT 
    s.cpf, 
    s.dt_mes as dt_mes_true, 
    s.periodo_antes_inicio, 
    s.periodo_antes_fim, 
    s.periodo_depois_inicio, 
    s.periodo_depois_fim, 
    s.vl_compra_antes_90d, 
    s.qt_compra_antes_90d, 
    s.dt_last_use_credit, 
    s.dt_first_use_credit, 
    s.dt_criacao_conta, 
    s.vl_current_limit, 
    c.*, 
    case when c.cpf_customer is null then 0 else 1 end as usou_chat 
  FROM 
    spending_antes_total s 
    LEFT JOIN chats_agg c on (
      s.cpf = c.cpf_customer 
      and c.dt_mes = s.dt_mes
    )
), 
spending_depois as (
  select 
    c.cpf, 
    c.dt_mes_true, 
    sum(s_60.vl_transacao) as vl_compra_depois_ano, 
    count(distinct s_60.id_autorizacao) as qt_compras_depois_ano 
  from 
    base_total_add_chats c 
    left join platform_curated_zone.transaction_dedicada_spending s_60 on (
      c.cpf = s_60.cpf 
      and cast(s_60.dt_dia as date) >= c.periodo_depois_inicio 
      and cast(s_60.dt_dia as date) <= c.periodo_depois_fim
    ) 
  group by 
    1, 
    2
), 
base_completa as (
  SELECT 
    a.cpf, 
    a.dt_mes_true, 
    a.qt_compra_antes_90d, 
    a.dt_last_use_credit, 
    a.vl_current_limit, 
    a.flag_problema, 
    a.nr_tpr, 
    a.nr_tma, 
    a.nr_tmat, 
    a.usou_chat, 
    a.hierarquia_1, 
    max(a.vl_compra_antes_90d) vl_compra_antes_90d, 
    min(a.dt_first_use_credit) dt_first_use_credit, 
    min(a.dt_criacao_conta) dt_criacao_conta, 
    count(
      distinct substring(
        cast(dt_event as varchar), 
        1, 
        10
      )
    ) as dias_uso_app, 
    max(d2.vl_compra_depois_ano) vl_compra_depois_ano, 
    max(d2.qt_compras_depois_ano) qt_compras_depois_ano 
  FROM 
    base_total_add_chats a 
    LEFT JOIN customer_curated_zone.ca_book_cliente b_c ON (a.cpf = b_c.cpf) 
    LEFT JOIN customer_curated_zone.ca_analitico_geoloc ap on (
      ap.cpf = a.cpf 
      and ap.dt_event >= a.periodo_antes_inicio 
      and ap.dt_event <= a.periodo_antes_fim
    ) 
    LEFT JOIN spending_depois d2 on (
      a.cpf = d2.cpf 
      and a.dt_mes_true = d2.dt_mes_true
    ) 
  where 
    a.usou_chat = 1 
  group by 
    1, 
    2, 
    3, 
    4, 
    5, 
    6, 
    7, 
    8, 
    9, 
    10, 
    11
) 
select 
  * 
from 
  base_completa



"""

## Calculando a taxa de churn por grupo

In [246]:
df_aa = ac.df_athena_q('flavia-costa', query_aa)

missing_zero = ['qt_compra_antes_90d', 'vl_compra_antes_90d', 'vl_compra_depois_ano']

for column in missing_zero:
    df_aa[column].fillna(0, inplace = True)

df_aa['dt_mes_true'] = pd.to_datetime(df_aa['dt_mes_true'])
df_aa['dt_last_use_credit'] = pd.to_datetime(df_aa['dt_last_use_credit'])
df_aa['dt_criacao_conta_str'] = df_aa['dt_criacao_conta'].astype('string') + '01'
df_aa['dt_criacao_conta_date'] = pd.to_datetime(df_aa['dt_criacao_conta_str'], format='%Y%m%d', errors='ignore')

# variaveis dias
df_aa['dias_desde_ultima_compra'] = (df_aa['dt_mes_true'] - df_aa['dt_last_use_credit']).dt.days 
df_aa['cc_time'] = (df_aa['dt_mes_true'] - df_aa['dt_criacao_conta_date']).dt.days 
df_aa['dias_ativacao'] = (df_aa['dt_mes_true'] - df_aa['dt_first_use_credit']).dt.days 

vars_rfm = ['dias_ativacao', 'cc_time', 'dias_desde_ultima_compra', 'qt_compra_antes_90d', 'vl_compra_antes_90d']

for column in vars_rfm:
  if df_aa[column].dtype == 'float' or df_aa[column].dtype == 'int':
    nm_col = column + "_fx"
    df_aa[nm_col] = pd.qcut(df_aa[column].rank(method='first'), q=3, precision=0, labels = ['A', 'B', 'C'])

df_aa['segmento'] = df_aa.cc_time_fx.astype("string") + df_aa.dias_desde_ultima_compra_fx.astype("string") + df_aa.qt_compra_antes_90d_fx.astype("string") + df_aa.vl_compra_antes_90d_fx.astype("string")
df_aa['churn_depois'] = np.where(df_aa.vl_compra_depois_ano > 0, 0, 1)
tabela_churn = df_aa[['churn_depois', 'segmento']].groupby('segmento').mean()
tabela_churn = tabela_churn.reset_index()
tabela_churn['LT'] = (1 / (tabela_churn['churn_depois']/12))

## Calcula Life time mediano dos clientes do recorte

In [247]:
tabela_churn['LT'].min()

22.000000000000004

In [248]:
tabela_churn['LT'].median()

36.514218932606155

In [249]:
tabela_churn['LT'].max()

67.41532813217071

In [250]:
lt_mediano= tabela_churn['LT'].median()

In [251]:
#MARCANDO O CHURN NA BASE ATUAL
for column in vars_rfm:
    if df[column].dtype == 'float' or df[column].dtype == 'int':
      nm_col = column + "_fx"
      df[nm_col] = pd.qcut(df[column].rank(method='first'), q=3, precision=0, labels = ['A', 'B', 'C'])

df['segmento'] = df.cc_time_fx.astype("string") + df.dias_desde_ultima_compra_fx.astype("string") + df.qt_compra_antes_90d_fx.astype("string") + df.vl_compra_antes_90d_fx.astype("string")
df = df.join(tabela_churn.set_index('segmento'), on = 'segmento', how = 'left' )

In [252]:
df.head()

,cpf,dt_mes_true,periodo_antes_inicio,periodo_antes_fim,periodo_depois_inicio,periodo_depois_fim,periodo_depois30_fim,qt_compra_antes_90d,dt_last_use_credit,vl_current_limit,...,dias_desde_ultima_compra,vl_compra_antes_90d_log,dias_ativacao_fx,cc_time_fx,dias_desde_ultima_compra_fx,qt_compra_antes_90d_fx,vl_compra_antes_90d_fx,segmento,churn_depois,LT
0,05308512450,2023-06-01,2023-03-02,2023-05-31,2023-07-01,2023-09-01,2023-08-01,4.0,2023-05-13,1400.0,...,19.0,5.670949,C,C,C,B,B,CCBB,0.324189,37.015442
1,08543541409,2023-03-01,2022-11-30,2023-02-28,2023-04-01,2023-06-01,2023-05-01,14.0,2023-01-30,1600.0,...,30.0,7.521047,C,C,C,C,C,CCCC,0.301006,39.866310
2,94578346404,2023-06-01,2023-03-02,2023-05-31,2023-07-01,2023-09-01,2023-08-01,7.0,2023-05-24,600.0,...,8.0,7.153693,A,A,A,B,C,AABC,0.308586,38.887070
3,54639433204,2023-04-01,2022-12-31,2023-03-31,2023-05-01,2023-07-01,2023-06-01,2.0,2023-03-08,500.0,...,24.0,6.198479,A,A,C,B,B,ACBB,0.389896,30.777409
4,12471873650,2023-05-01,2023-01-30,2023-04-30,2023-06-01,2023-08-01,2023-07-01,3.0,2023-03-23,3200.0,...,39.0,7.897326,C,C,C,B,C,CCBC,0.252689,47.489184


In [253]:
df['LT'].median()

34.00435729847494

## Parametros modelo churn

In [254]:
#Parametros modelo - regressão logística do estudo prévio
c_intercepto = -0.1635
c_usou_chat = -0.0655
c_flag_problema = -0.0993
c_vl_compra_antes_90d_log = 0.1318
c_cc_time = -0.6206
c_vl_current_limit = 0.3866
c_dias_uso_app = 0.2392
c_vl_renda_declarada = -5.3729
c_dias_ativacao = 0.3947
c_qt_compra_antes_90d = 0.2761
c_dias_desde_ultima_compra = -2.9933

queda_cr = 0.1

Loop simulações

In [255]:
#iteracoes = 500

In [256]:
#df_simulacoes = pd.DataFrame()

## Inputando score de churn pra todos

In [257]:
df['usou_chat_sim'] = 0
df['flag_problema_sim'] = 0

df['rawPrediction'] = c_intercepto  + ((df.usou_chat_sim * c_usou_chat) + (df.flag_problema_sim * c_flag_problema) 
  + (df.vl_compra_antes_90d_log * c_vl_compra_antes_90d_log) 
  + (df_n.cc_time * c_cc_time)
  + (df_n.vl_current_limit * c_vl_current_limit) 
  + (df_n.dias_uso_app * c_dias_uso_app) 
  + (df_n.vl_renda_declarada * c_vl_renda_declarada) 
  + (df_n.dias_ativacao * c_dias_ativacao) 
  + (df_n.qt_compra_antes_90d * c_qt_compra_antes_90d)
  + (df_n.dias_desde_ultima_compra * c_dias_desde_ultima_compra))

df['score_compra_depois']  =  1/(1 + np.exp(-1 * df.rawPrediction))
df['score_compra_depois'] = np.where(df['score_compra_depois'] < 0, 0, df['score_compra_depois'])
df['score_compra_depois'] = np.where(df['score_compra_depois'] > 1, 1, df['score_compra_depois'])

In [258]:
df['score_compra_depois'].median()

0.5183409817841954

## Levantamento de clientes no recorte que entraram em churn (não compraram após 2 meses)

In [259]:
df['teve_compra_depois'] = np.where(df.vl_compra_depois_60d > 0 , 1, 0)
df['teve_compra_depois'].value_counts()

1    246173
0    181322
Name: teve_compra_depois, dtype: int64

In [260]:
df_agg_recorte = df.groupby(['dt_mes_true', 'hierarquia_1', 'hierarquia_4']).agg({'cpf':['nunique']})
df_agg_recorte.columns = df_agg_recorte.columns.droplevel(0)
df_agg_recorte = df_agg_recorte.reset_index()
df_agg_recorte.columns = ['dt_mes', 'hierarquia_1', 'hierarquia_4', 'clientes_recorte_total']
df_agg_recorte.head()

,dt_mes,hierarquia_1,hierarquia_4,clientes_recorte_total
0,2023-01-01,Assunto não identificado,Assunto não identificado,567
1,2023-01-01,Atendimento,Atendimento de terceiros,208
2,2023-01-01,Atendimento,Canais de atendimento,67
3,2023-01-01,Atendimento,Chat duplicado,12
4,2023-01-01,Atendimento,Chat sem retorno,964


In [261]:
df_agg_churn = df.query('teve_compra_depois == 0').groupby(['dt_mes_true', 'hierarquia_1', 'hierarquia_4']).agg({'cpf':['nunique']})
df_agg_churn.columns = df_agg_churn.columns.droplevel(0)
df_agg_churn = df_agg_churn.reset_index()
df_agg_churn.columns = ['dt_mes', 'hierarquia_1', 'hierarquia_4', 'clientes_recorte_churn']
df_agg_churn.head()


,dt_mes,hierarquia_1,hierarquia_4,clientes_recorte_churn
0,2023-01-01,Assunto não identificado,Assunto não identificado,77
1,2023-01-01,Atendimento,Atendimento de terceiros,22
2,2023-01-01,Atendimento,Canais de atendimento,12
3,2023-01-01,Atendimento,Chat duplicado,1
4,2023-01-01,Atendimento,Chat sem retorno,133


In [262]:
df_count_cx.head()

,dt_mes,hierarquia_1,hierarquia_4,clientes_cx
0,2023-05-01,Cartão,Rastreio de cartão,5863
1,2023-06-01,Cartão,Informações sobre fatura,4803
2,2023-03-01,Cartão,Token E-mail,1731
3,2023-02-01,Conta,Gerar fatura,116
4,2023-02-01,Conta,Dúvidas em lançamentos,7


### Definindo qtos clientes já entrariam em churn independente do atendimento

In [263]:
df['fx_score'] = np.round(df['score_compra_depois'],1)
  
#tabela KS
agg_df_compra_sim = df[df['teve_compra_depois'] == 1][['fx_score', 'cpf']].groupby('fx_score').count()
agg_df_compra_nao = df[df['teve_compra_depois'] == 0][['fx_score', 'cpf']].groupby('fx_score').count()
agg_df_compra = agg_df_compra_sim.join(agg_df_compra_nao, lsuffix='nao', rsuffix='sim')
agg_df_compra = agg_df_compra.reset_index()
agg_df_compra['cumperc_nao'] = agg_df_compra['cpfnao'].cumsum()/agg_df_compra['cpfnao'].sum()*100
agg_df_compra['cumperc_sim'] = agg_df_compra['cpfsim'].cumsum()/agg_df_compra['cpfsim'].sum()*100
agg_df_compra['dif'] = agg_df_compra['cumperc_nao'] - agg_df_compra['cumperc_sim']
agg_df_compra['dif_abs'] = np.abs(agg_df_compra['dif']) 

#quando a diferença nas distribuições ficar acima de 10%
dif_maxima = agg_df_compra['dif_abs'].max()
threshould_conservador = agg_df_compra[agg_df_compra.dif_abs == dif_maxima]['fx_score'].min()

df['compra_depois_propensao']  = np.where(df['fx_score'] > threshould_conservador, 1,0)

#inputando spending para o cliente que teve churn de fato, que foi sorteado para não ter atendimento na simulação e com baixo score de compra depois
df['flag_atribui_spending'] = np.where((df['compra_depois_propensao'] == 1) & (df['teve_compra_depois'] == 0) & (df.usou_chat_sim == 0),1,0)

In [264]:
df['flag_atribui_spending'].value_counts()

0    393834
1     33661
Name: flag_atribui_spending, dtype: int64

In [265]:
df_agg_churn_recuperados = df.query('flag_atribui_spending == 1').groupby(['dt_mes_true', 'hierarquia_1', 'hierarquia_4']).agg({'cpf':['nunique']})
df_agg_churn_recuperados.columns = df_agg_churn_recuperados.columns.droplevel(0)
df_agg_churn_recuperados = df_agg_churn_recuperados.reset_index()
df_agg_churn_recuperados.columns = ['dt_mes', 'hierarquia_1', 'hierarquia_4', 'clientes_recorte_churn_recuperados']
df_agg_churn_recuperados.head()

,dt_mes,hierarquia_1,hierarquia_4,clientes_recorte_churn_recuperados
0,2023-01-01,Assunto não identificado,Assunto não identificado,65
1,2023-01-01,Atendimento,Atendimento de terceiros,19
2,2023-01-01,Atendimento,Canais de atendimento,11
3,2023-01-01,Atendimento,Chat sem retorno,117
4,2023-01-01,Cartão,Alteração de Cadastro,334


### Estimando o gasto que os recuperados teriam nos 60 dias com base no perfil

In [266]:
df_comprou = df.query('teve_compra_depois == 1')
df_nao_comprou = df.query('flag_atribui_spending == 1')

In [267]:
for column in df_nao_comprou.columns:
    if df_nao_comprou[column].dtype == 'object':
        df_nao_comprou[column].fillna(df_nao_comprou[column].mode()[0], inplace = True)
    elif df_nao_comprou[column].dtype == 'string':
        pass
    elif df_nao_comprou[column].dtype == 'category':
        pass
    elif df_nao_comprou[column].dtype == 'datetime64[ns]':
        pass
    elif df_nao_comprou[column].dtype != 'object':
        df_nao_comprou[column].fillna(df_nao_comprou[column].median(), inplace = True)

/tmp/ipykernel_134303/710299964.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nao_comprou[column].fillna(df_nao_comprou[column].mode()[0], inplace = True)
/tmp/ipykernel_134303/710299964.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nao_comprou[column].fillna(df_nao_comprou[column].mode()[0], inplace = True)
/tmp/ipykernel_134303/710299964.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nao_comprou[column].fill

#### Modelo linear

In [268]:
formula_text = ("""
      vl_compra_depois_60d ~ 
      C(periodo_depois30_fim) + 
        qt_compra_antes_90d +  vl_current_limit + 
        cc_time + dias_ativacao + dias_desde_ultima_compra +
       vl_compra_antes_90d + vl_renda_declarada  + dias_uso_app
      """)

model = ols(formula = formula_text, data = df_comprou).fit()

In [269]:
prediction = model.predict(df_nao_comprou)
df_nao_comprou['valor incremental bruto'] = prediction

/tmp/ipykernel_134303/829968082.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nao_comprou['valor incremental bruto'] = prediction


In [270]:
df_nao_comprou['valor incremental bruto'].median()

1214.21129407836

#### Ajustando o valor incremental bruto para o caso de ser maior que o limite do cliente

In [271]:
df_nao_comprou['valor incremental bruto'] = np.where(df_nao_comprou['valor incremental bruto'] >  df_nao_comprou['vl_current_limit'], df_nao_comprou['vl_current_limit'], df_nao_comprou['valor incremental bruto'])

/tmp/ipykernel_134303/4211581140.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nao_comprou['valor incremental bruto'] = np.where(df_nao_comprou['valor incremental bruto'] >  df_nao_comprou['vl_current_limit'], df_nao_comprou['vl_current_limit'], df_nao_comprou['valor incremental bruto'])


### Estimando LTV

In [272]:
df_nao_comprou['LTV Receita'] = np.where(df_nao_comprou['ltv'] < 0, 0, df_nao_comprou['ltv'] * df_nao_comprou['LT'])

/tmp/ipykernel_134303/1633370652.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nao_comprou['LTV Receita'] = np.where(df_nao_comprou['ltv'] < 0, 0, df_nao_comprou['ltv'] * df_nao_comprou['LT'])


In [273]:
df_agg_vl_bruto_recuperados = df_nao_comprou.groupby(['dt_mes_true', 'hierarquia_1', 'hierarquia_4']).agg({'valor incremental bruto':['sum'], 'LTV Receita':['sum'], 'LT':['median']})
df_agg_vl_bruto_recuperados.columns = df_agg_vl_bruto_recuperados.columns.droplevel(0)
df_agg_vl_bruto_recuperados = df_agg_vl_bruto_recuperados.reset_index()
df_agg_vl_bruto_recuperados.columns = ['dt_mes', 'hierarquia_1', 'hierarquia_4', 'vl_bruto_recuperados_60d', 'LTV_Receita_Recuperado', 'LT_mediano_recuperados']
df_agg_vl_bruto_recuperados.head()

,dt_mes,hierarquia_1,hierarquia_4,vl_bruto_recuperados_60d,LTV_Receita_Recuperado,LT_mediano_recuperados
0,2023-01-01,Assunto não identificado,Assunto não identificado,72517.104265,9.132207e+04,30.777409
1,2023-01-01,Atendimento,Atendimento de terceiros,27835.689931,1.318349e+05,37.016801
2,2023-01-01,Atendimento,Canais de atendimento,11684.054618,6.142314e+03,37.015442
3,2023-01-01,Atendimento,Chat sem retorno,230524.163274,9.258041e+05,39.866310
4,2023-01-01,Cartão,Alteração de Cadastro,433944.700386,1.083040e+06,37.015442


### LTV Total para relativizar

### Unindo bases

In [274]:
df_agg_vl_bruto_recuperados['dt_mes'] = df_agg_vl_bruto_recuperados['dt_mes'].astype('object')
df_agg_churn_recuperados['dt_mes'] = df_agg_churn_recuperados['dt_mes'].astype('object')
df_agg_churn['dt_mes'] = df_agg_churn['dt_mes'].astype('object')


In [275]:
df_count_cx['dt_mes'] = df_count_cx['dt_mes'].astype('object')

In [276]:
df_agg_recorte['dt_mes'] = df_agg_recorte['dt_mes'].astype('object')

In [277]:
df_count_cx['chave'] = df_count_cx.dt_mes.astype('string') + df_count_cx.hierarquia_1.astype('string') + df_count_cx.hierarquia_4.astype('string')
df_agg_vl_bruto_recuperados['chave'] = df_agg_vl_bruto_recuperados.dt_mes.astype('string') + df_agg_vl_bruto_recuperados.hierarquia_1.astype('string') + df_agg_vl_bruto_recuperados.hierarquia_4.astype('string')
df_agg_churn_recuperados['chave'] = df_agg_churn_recuperados.dt_mes.astype('string') + df_agg_churn_recuperados.hierarquia_1.astype('string') + df_agg_churn_recuperados.hierarquia_4.astype('string')
df_agg_churn['chave'] = df_agg_churn.dt_mes.astype('string') + df_agg_churn.hierarquia_1.astype('string') + df_agg_churn.hierarquia_4.astype('string')
df_agg_recorte['chave'] = df_agg_recorte.dt_mes.astype('string') + df_agg_recorte.hierarquia_1.astype('string') + df_agg_recorte.hierarquia_4.astype('string')


In [278]:
df_count_cx['chave'] = df_count_cx['chave'].str.replace(' 00:00:00', '')
df_agg_vl_bruto_recuperados['chave'] = df_agg_vl_bruto_recuperados['chave'].str.replace(' 00:00:00', '')
df_agg_churn_recuperados['chave'] = df_agg_churn_recuperados['chave'].str.replace(' 00:00:00', '')
df_agg_churn['chave'] = df_agg_churn['chave'].str.replace(' 00:00:00', '')
df_agg_recorte['chave'] = df_agg_recorte['chave'].str.replace(' 00:00:00', '')

In [279]:
df_agg_vl_bruto_recuperados = df_agg_vl_bruto_recuperados.drop(['dt_mes', 'hierarquia_1', 'hierarquia_4'], axis=1)
df_agg_churn_recuperados = df_agg_churn_recuperados.drop(['dt_mes', 'hierarquia_1', 'hierarquia_4'], axis=1)
df_agg_churn = df_agg_churn.drop(['dt_mes', 'hierarquia_1', 'hierarquia_4'], axis=1)
df_agg_recorte = df_agg_recorte.drop(['dt_mes', 'hierarquia_1', 'hierarquia_4'], axis=1)

In [280]:
df_unificado = pd.merge(df_count_cx, df_agg_vl_bruto_recuperados, on ='chave', how = 'left')
df_unificado = pd.merge(df_unificado, df_agg_recorte, on ='chave', how = 'left')
df_unificado = pd.merge(df_unificado, df_agg_churn, on ='chave', how = 'left')
df_unificado = pd.merge(df_unificado, df_agg_churn_recuperados, on ='chave', how = 'left')

In [281]:
df_unificado = df_unificado.fillna(0.0)

In [282]:
df_unificado

,dt_mes,hierarquia_1,hierarquia_4,clientes_cx,chave,vl_bruto_recuperados_60d,LTV_Receita_Recuperado,LT_mediano_recuperados,clientes_recorte_total,clientes_recorte_churn,clientes_recorte_churn_recuperados
0,2023-05-01,Cartão,Rastreio de cartão,5863,2023-05-01CartãoRastreio de cartão,111936.330526,2.839536e+04,38.887070,690.0,75.0,67.0
1,2023-06-01,Cartão,Informações sobre fatura,4803,2023-06-01CartãoInformações sobre fatura,314819.501130,1.140602e+06,37.015442,1081.0,190.0,155.0
2,2023-03-01,Cartão,Token E-mail,1731,2023-03-01CartãoToken E-mail,37688.014983,2.524780e+05,37.015442,283.0,42.0,35.0
3,2023-02-01,Conta,Gerar fatura,116,2023-02-01ContaGerar fatura,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0
4,2023-02-01,Conta,Dúvidas em lançamentos,7,2023-02-01ContaDúvidas em lançamentos,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1877,2023-01-01,Conta,Problema de Cadastro da proposta,2,2023-01-01ContaProblema de Cadastro da proposta,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0
1878,2023-04-01,Fraude,Dúvidas sobre empréstimo/crédito pessoal,1,2023-04-01FraudeDúvidas sobre empréstimo/crédi...,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0
1879,2023-04-01,Marketing,Rastreio de cartão,5,2023-04-01MarketingRastreio de cartão,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0
1880,2023-02-01,Fraude,Dúvidas sobre empréstimo/crédito pessoal,4,2023-02-01FraudeDúvidas sobre empréstimo/crédi...,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0


In [283]:
#df_unificado.to_csv('analise_tabela.csv')

## LTV Geral

In [284]:
query_ltv = """

select
mesref
, sum(ltv) as montante_ltv
, avg(ltv) as media_ltv
, approx_percentile(ltv, 0.5) as mediana_ltv
,  count(distinct cpf) as clientes_rentaveis_total
from public.rentabilidade_cartoes_diego_camilo
where ltv > 0
group by 1
order by mesref


"""

In [285]:
ltv_rentaveis = ac.df_athena_q('flavia-costa', query_ltv)

In [286]:
ltv_rentaveis['dt_mes'] = ltv_rentaveis['mesref'] + '-01'
ltv_rentaveis['dt_mes'] = pd.to_datetime(ltv_rentaveis['dt_mes'])

In [287]:
df_unificado['dt_mes'] = pd.to_datetime(df_unificado['dt_mes'])

In [288]:
df_unificado_final = pd.merge(df_unificado, ltv_rentaveis[['dt_mes','montante_ltv','media_ltv','mediana_ltv','clientes_rentaveis_total']], on = 'dt_mes', how = 'left')

In [289]:
df_unificado_final['montante_ltv'] = df_unificado_final['montante_ltv'] * df_unificado_final['LT_mediano_recuperados']
df_unificado_final['media_ltv'] = df_unificado_final['media_ltv'] * df_unificado_final['LT_mediano_recuperados']
df_unificado_final['mediana_ltv'] = df_unificado_final['mediana_ltv'] * df_unificado_final['LT_mediano_recuperados']

In [290]:
df_unificado_final['perc_recorte'] = np.where(df_unificado_final['clientes_cx'] > 0, df_unificado_final['clientes_recorte_total'] / df_unificado_final['clientes_cx'],0)

df_unificado_final['perc_churn_recorte'] = np.where(df_unificado_final['clientes_recorte_total'] > 0, df_unificado_final['clientes_recorte_churn'] / df_unificado_final['clientes_recorte_total'], 0)

df_unificado_final['perc_recuperacao_churn'] = np.where(df_unificado_final['clientes_recorte_churn'] > 0, df_unificado_final['clientes_recorte_churn_recuperados'] / df_unificado_final['clientes_recorte_churn'], 0)

In [291]:
df_unificado_final

,dt_mes,hierarquia_1,hierarquia_4,clientes_cx,chave,vl_bruto_recuperados_60d,LTV_Receita_Recuperado,LT_mediano_recuperados,clientes_recorte_total,clientes_recorte_churn,clientes_recorte_churn_recuperados,montante_ltv,media_ltv,mediana_ltv,clientes_rentaveis_total,perc_recorte,perc_churn_recorte,perc_recuperacao_churn
0,2023-05-01,Cartão,Rastreio de cartão,5863,2023-05-01CartãoRastreio de cartão,111936.330526,2.839536e+04,38.887070,690.0,75.0,67.0,4.566259e+09,2585.851390,425.011614,1765592,0.117687,0.108696,0.893333
1,2023-06-01,Cartão,Informações sobre fatura,4803,2023-06-01CartãoInformações sobre fatura,314819.501130,1.140602e+06,37.015442,1081.0,190.0,155.0,5.059187e+09,2733.903827,378.903888,1850274,0.225068,0.175763,0.815789
2,2023-03-01,Cartão,Token E-mail,1731,2023-03-01CartãoToken E-mail,37688.014983,2.524780e+05,37.015442,283.0,42.0,35.0,3.698781e+09,2331.565283,399.130917,1586123,0.163489,0.148410,0.833333
3,2023-02-01,Conta,Gerar fatura,116,2023-02-01ContaGerar fatura,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000e+00,0.000000,0.000000,1492153,0.000000,0.000000,0.000000
4,2023-02-01,Conta,Dúvidas em lançamentos,7,2023-02-01ContaDúvidas em lançamentos,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000e+00,0.000000,0.000000,1492153,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1877,2023-01-01,Conta,Problema de Cadastro da proposta,2,2023-01-01ContaProblema de Cadastro da proposta,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000e+00,0.000000,0.000000,1448099,0.000000,0.000000,0.000000
1878,2023-04-01,Fraude,Dúvidas sobre empréstimo/crédito pessoal,1,2023-04-01FraudeDúvidas sobre empréstimo/crédi...,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000e+00,0.000000,0.000000,1656724,0.000000,0.000000,0.000000
1879,2023-04-01,Marketing,Rastreio de cartão,5,2023-04-01MarketingRastreio de cartão,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000e+00,0.000000,0.000000,1656724,0.000000,0.000000,0.000000
1880,2023-02-01,Fraude,Dúvidas sobre empréstimo/crédito pessoal,4,2023-02-01FraudeDúvidas sobre empréstimo/crédi...,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000e+00,0.000000,0.000000,1492153,0.000000,0.000000,0.000000


## Exporta para Sandbox

In [292]:
area = 'cx'
name = 'report_perdas_cr.csv'
filename = f'{area}.{name}'

df_unificado_final.drop('chave', axis = 1).to_csv(filename, index=False)

os.system(f'aws s3 cp {filename} s3://data-sandbox-zone-will-prod/upload/')

upload: ./cx.report_perdas_cr.csv to s3://data-sandbox-zone-will-prod/upload/cx.report_perdas_cr.csv


0